In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
import tensorflow as tf

In [2]:
data_train = pd.read_csv('../train_data.csv').values
data_test = pd.read_csv('../test_data.csv').values

# Build X and y
X_train = data_train[:, :-1]
y_train = data_train[:, -1]
X_test = data_test[:, :-1]
y_test = data_test[:, -1]

# Number of stocks in training data
n_att = X_train.shape[1]

# Neurons
n_neurons_1 = 1024
n_neurons_2 = 512
n_neurons_3 = 256
n_neurons_4 = 128

# Placeholder
X = tf.placeholder(dtype=tf.float32, shape=[None, n_att])
Y = tf.placeholder(dtype=tf.float32, shape=[None])

# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

# Hidden weights
W_hidden_1 = tf.Variable(weight_initializer([n_att, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

# Output weights
W_out = tf.Variable(weight_initializer([n_neurons_4, 1]))
bias_out = tf.Variable(bias_initializer([1]))

# Hidden layer
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

# Output layer (transpose!)
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out))

# Cost function
mse = tf.reduce_mean(tf.squared_difference(out, Y))

# Optimizer
opt = tf.train.AdamOptimizer().minimize(mse)

# Saver
saver = tf.train.Saver()

In [3]:
with tf.Session() as sess:
    
    # Init
    sess.run(tf.global_variables_initializer())

    # Fit neural net
    batch_size = 256
    mse_train = []
    mse_test = []

    # Run
    epochs = 20
    for e in range(epochs):

        # Shuffle training data
        shuffle_indices = np.random.permutation(np.arange(len(y_train)))
        X_train = X_train[shuffle_indices]
        y_train = y_train[shuffle_indices]

        # Minibatch training
        for i in range(0, len(y_train) // batch_size):
            start = i * batch_size
            batch_x = X_train[start:start + batch_size]
            batch_y = y_train[start:start + batch_size]
            # Run optimizer with batch
            sess.run(opt, feed_dict={X: batch_x, Y: batch_y})

            # Show progress
            if np.mod(i, 50) == 0:
                # MSE train and test
                mse_train.append(sess.run(mse, feed_dict={X: X_train, Y: y_train}))
                mse_test.append(sess.run(mse, feed_dict={X: X_test, Y: y_test}))
                print('MSE Train: ', mse_train[-1])
                print('MSE Test: ', mse_test[-1])
                # Prediction
                pred = sess.run(out, feed_dict={X: X_test})
                print('pred: ', pred)
    save_path = saver.save(sess, "../models/model.ckpt")
    print("Model saved in path: %s" % save_path)

MSE Train:  0.7505675
MSE Test:  0.76417184
pred:  [[0.2426271  0.5484703  0.678931   ... 0.255826   0.32285464 0.4054969 ]]
MSE Train:  0.0057190596
MSE Test:  0.0059631346
pred:  [[0.76740515 1.317698   2.3443074  ... 0.0982063  0.48812765 0.45308706]]
MSE Train:  0.0057486356
MSE Test:  0.0059463764
pred:  [[0.9741066  1.3745159  2.5315695  ... 0.13357843 0.51309514 0.47433925]]
MSE Train:  0.0022031262
MSE Test:  0.0025126722
pred:  [[0.8639091  1.2652779  2.3047166  ... 0.07050347 0.46016932 0.4236863 ]]
MSE Train:  0.002552236
MSE Test:  0.002855581
pred:  [[0.81838125 1.2696753  2.317388   ... 0.11193291 0.4304467  0.44746482]]
MSE Train:  0.0016493307
MSE Test:  0.0020786182
pred:  [[0.9328684  1.2862482  2.3412273  ... 0.12112416 0.47505462 0.42744288]]
MSE Train:  0.0023999522
MSE Test:  0.0028119392
pred:  [[0.8267211  1.2921579  2.3509     ... 0.0612753  0.42412853 0.37152696]]
MSE Train:  0.0014742027
MSE Test:  0.0018141991
pred:  [[0.876138   1.2821375  2.4602304  ... 0.